In [3]:
import mlrun

In [4]:
# load function from Github
xfn = mlrun.import_function('https://raw.githubusercontent.com/yjb-ds/functions/master/fileutils/function.yaml')

# configute it: mount on iguazio fabric, set as interactive (return stdout)
xfn.apply(mlrun.mount_v3io())
xfn.interactive = True

# create and run the task

images_path = '/User/mlrun/functions/images'

open_archive_task = mlrun.NewTask(
    'download',
    handler='open_archive', 
    params={'target_path': images_path},
    inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'}
)

# run
run = xfn.run(open_archive_task)

/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[mlrun] 2020-01-09 21:26:43,706 starting run download uid=1c07bbc86bee4f10991bf5262bf544ea  -> 
[mlrun] 2020-01-09 21:26:43,708 warning!, Api url not set, trying to exec remote runtime locally
[mlrun] 2020-01-09 21:26:43,710 using in-cluster config.
[mlrun] 2020-01-09 21:26:43,725 Pod download-rmclp created
[mlrun] 2020-01-09 21:26:43,726 Job is running in the background, pod: download-rmclp


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...f544ea,0,,completed,download,kind=jobowner=admin,archive_url,target_path=/User/mlrun/functions/images,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 1c07bbc86bee4f10991bf5262bf544ea  , !mlrun logs 1c07bbc86bee4f10991bf5262bf544ea 
[mlrun] 2020-01-09 21:26:43,776 run executed, status=completed


#### load and configure function

In [5]:
# load function from Github
xfn = mlrun.import_function('https://raw.githubusercontent.com/yjb-ds/functions/arc2parq/fileutils/arc_to_parquet.yaml')

# configure function: mount on Iguazio data fabric, set as interactive (return stdout)
xfn.apply(mlrun.mount_v3io())
xfn.interactive = True

/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [6]:
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

#### deploy / build

The following triggers a build when run for the first time using specs found in the yaml file above:

In [7]:
xfn.deploy()

[mlrun] 2020-01-09 21:26:44,403 starting remote build, image: .mlrun/func-default-arc-to-parquet-latest
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:0318d80cb241983eda20b905d77fa0bfb06e29e5aabf075c7941ea687f1c125a: no such file or directory 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:0318d80cb241983eda20b905d77fa0bfb06e29e5aabf075c7941ea687f1c125a: no such file or directory 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0001] Unpacking rootfs as cmd RUN pip install -q mlrun requires it. 
INFO[0011] Taking snapshot of f

True

In [8]:
# create and run the task
target_path = '/User/mlrun/functions/parquet'
archive = 'https://fpsignals-public.s3.amazonaws.com/x_test_50.csv.gz'

arc_to_parq_task = mlrun.NewTask(
    'arc2parq', 
    handler='arc_to_parquet',
    params={
        'target_path': target_path,
        'name'       : 'x_test_50.parquet', # the file extension is not necessary
        'key'        : 'raw_data',
        'archive_url': archive
    })

# run
run = xfn.run(arc_to_parq_task)

[mlrun] 2020-01-09 21:28:47,515 starting run arc2parq uid=ed20cbdcddb3473882507594f69e6180  -> http://mlrun-api:8080
[mlrun] 2020-01-09 21:29:03,735 destination file does not exist, downloading
[mlrun] 2020-01-09 21:29:03,873 saved table to /User/mlrun/functions/parquet/x_test_50.parquet
[mlrun] 2020-01-09 21:29:03,873 logging /User/mlrun/functions/parquet/x_test_50.parquet to context

[mlrun] 2020-01-09 21:29:03,898 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...9e6180,0,Jan 09 21:29:03,completed,arc-to-parquet,host=arc2parq-n87rgkind=jobowner=admin,,archive_url=https://fpsignals-public.s3.amazonaws.com/x_test_50.csv.gzkey=raw_dataname=x_test_50.parquettarget_path=/User/mlrun/functions/parquet,,raw_data


to track results use .show() or .logs() or in CLI: 
!mlrun get run ed20cbdcddb3473882507594f69e6180  , !mlrun logs ed20cbdcddb3473882507594f69e6180 
[mlrun] 2020-01-09 21:29:06,867 run executed, status=completed


___

In [9]:
import numpy as np
import pandas as pd

In [13]:
run.__dict__

{'_spec': <mlrun.model.RunSpec at 0x7f3a98280828>,
 '_metadata': <mlrun.model.RunMetadata at 0x7f3a990d4e10>,
 '_status': <mlrun.model.RunStatus at 0x7f3a98a04e80>}

In [11]:
original = pd.read_csv(archive)
copied   = pd.read_parquet(target_path+"/x_test_50.parquet", engine="pyarrow")
assert np.array_equal(original, copied), "original and copied data not equal"

In [ ]:
original = 